<a href="https://colab.research.google.com/github/D-K-Deng/Protein_Optimize/blob/main/Protein_Optimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Main Process
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install -q biopython
from google.colab import files
import os
import re
import hashlib
import random
import pandas as pd
from sys import version_info


python_version = f"{version_info.major}.{version_info.minor}"
seq_test="MCCNRGKNVSIENLHQGFTHIFESTFESTEGVAEYVSHPSHVEYANLFLANLEKVLVIDYKPTTVRV" #@param {type:"string"}
hr = ["R", "D", "H", "N", "E", "Q", "K"] #@param hr {type:"string", description:"hydrophobic residues that need to be changed, write in list format"}
ranges=[(11,25),(29,37),(39,56)]#@param hr {type:"string", description:"helices locations, write in list format, this is an example of location 11-25 29-37 39-56"}
results=[]
TL=[]
TH=[]
PC=[]
HM=[]
NHM=[]

#Sequence Generation

def get_actual_ranges(ranges, length):
    """Compute the actual replacement ranges considering boundaries and overlaps."""
    actual_ranges = []
    for start, end in ranges:
        actual_start = max(1, start - 5)
        actual_end = min(length, end + 5)
        actual_ranges.append((actual_start, actual_end))

    # Merge overlapping ranges
    merged_ranges = []
    sorted_ranges = sorted(actual_ranges, key=lambda x: x[0])
    for r in sorted_ranges:
        if not merged_ranges or r[0] > merged_ranges[-1][1]:
            merged_ranges.append(r)
        else:
            merged_ranges[-1] = (merged_ranges[-1][0], max(merged_ranges[-1][1], r[1]))

    return merged_ranges

def replace_in_range(sequence, start, end, replace_chars):
    """Find all possible replacements in the given range."""
    # Find positions of replaceable characters in the given range
    replace_positions = [i for i in range(start, end + 1) if sequence[i-1] in replace_chars]

    results = []
    if not replace_positions:
        return [sequence]

    for pos in replace_positions:
        # Replace one character
        new_seq = sequence[:pos-1] + 'K' + sequence[pos:]
        results.append(new_seq)

        # Replace two characters
        for sec_pos in replace_positions:
            if sec_pos > pos:
                new_seq_2 = new_seq[:sec_pos-1] + 'K' + new_seq[sec_pos:]
                results.append(new_seq_2)

    return results

def generate_replacements(sequence, ranges, replace_chars):
    """Generate all possible replacements for the given sequence."""
    actual_ranges = get_actual_ranges(ranges, len(sequence))
    results = [sequence]

    for start, end in actual_ranges:
        new_results = []
        for seq in results:
            new_results.extend(replace_in_range(seq, start, end, replace_chars))
        results = new_results

    return results

ans=generate_replacements(seq_test, ranges, hr)





def Recursion_Seq(seq_str):
  import os

  def add_hash(x,y):
    return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]
  query_sequence = seq_str
  jobname = 'test'
  # number of models to use
  num_relax = 1
  template_mode = "none"


  use_amber = num_relax > 0

  # remove whitespaces
  query_sequence = "".join(query_sequence.split())
  basejobname = "".join(jobname.split())
  basejobname = re.sub(r'\W+', '', basejobname)
  jobname = add_hash(basejobname, query_sequence)

  # check if directory with jobname exists
  def check(folder):
    if os.path.exists(folder):
      return False
    else:
      return True
  if not check(jobname):
    n = 0
    while not check(f"{jobname}_{n}"): n += 1
    jobname = f"{jobname}_{n}"

  # make directory to save results
  os.makedirs(jobname, exist_ok=True)

  # save queries
  queries_path = os.path.join(jobname, f"{jobname}.csv")
  with open(queries_path, "w") as text_file:
    text_file.write(f"id,sequence\n{jobname},{query_sequence}")

  if template_mode == "pdb100":
    use_templates = True
    custom_template_path = None
  elif template_mode == "custom":
    custom_template_path = os.path.join(jobname,f"template")
    os.makedirs(custom_template_path, exist_ok=True)
    uploaded = files.upload()
    use_templates = True
    for fn in uploaded.keys():
      os.rename(fn,os.path.join(custom_template_path,fn))
  else:
    custom_template_path = None
    use_templates = False

  print("jobname",jobname)
  print("sequence",query_sequence)
  print("length",len(query_sequence.replace(":","")))
  #@title Install dependencies
  import os
  USE_AMBER = use_amber
  USE_TEMPLATES = use_templates
  PYTHON_VERSION = python_version

  if not os.path.isfile("COLABFOLD_READY"):
    print("installing colabfold...")
    os.system("pip install -q --no-warn-conflicts 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'")
    os.system("pip install --upgrade dm-haiku")
    os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold")
    os.system("ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold")
    # patch for jax > 0.3.25
    os.system("sed -i 's/weights = jax.nn.softmax(logits)/logits=jnp.clip(logits,-1e8,1e8);weights=jax.nn.softmax(logits)/g' alphafold/model/modules.py")
    os.system("touch COLABFOLD_READY")

  if USE_AMBER or USE_TEMPLATES:
    if not os.path.isfile("CONDA_READY"):
      print("installing conda...")
      os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh")
      os.system("bash Mambaforge-Linux-x86_64.sh -bfp /usr/local")
      os.system("mamba config --set auto_update_conda false")
      os.system("touch CONDA_READY")

  if USE_TEMPLATES and not os.path.isfile("HH_READY") and USE_AMBER and not os.path.isfile("AMBER_READY"):
    print("installing hhsuite and amber...")
    os.system(f"mamba install -y -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 openmm=7.7.0 python='{PYTHON_VERSION}' pdbfixer")
    os.system("touch HH_READY")
    os.system("touch AMBER_READY")
  else:
    if USE_TEMPLATES and not os.path.isfile("HH_READY"):
      print("installing hhsuite...")
      os.system(f"mamba install -y -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python='{PYTHON_VERSION}'")
      os.system("touch HH_READY")
    if USE_AMBER and not os.path.isfile("AMBER_READY"):
      print("installing amber...")
      os.system(f"mamba install -y -c conda-forge openmm=7.7.0 python='{PYTHON_VERSION}' pdbfixer")
      os.system("touch AMBER_READY")
  msa_mode = "single_sequence"
  pair_mode = "unpaired_paired"
  # decide which a3m to use
  if "mmseqs2" in msa_mode:
    a3m_file = os.path.join(jobname,f"{jobname}.a3m")
  elif msa_mode == "custom":
    a3m_file = os.path.join(jobname,f"{jobname}.custom.a3m")
    if not os.path.isfile(a3m_file):
      custom_msa_dict = files.upload()
      custom_msa = list(custom_msa_dict.keys())[0]
      header = 0
      import fileinput
      for line in fileinput.FileInput(custom_msa,inplace=1):
        if line.startswith(">"):
          header = header + 1
        if not line.rstrip():
          continue
        if line.startswith(">") == False and header == 1:
          query_sequence = line.rstrip()
        print(line, end='')

      os.rename(custom_msa, a3m_file)
      queries_path=a3m_file
      print(f"moving {custom_msa} to {a3m_file}")

  else:
    a3m_file = os.path.join(jobname,f"{jobname}.single_sequence.a3m")
    with open(a3m_file, "w") as text_file:
      text_file.write(">1\n%s" % query_sequence)

  model_type = "alphafold2_ptm"
  num_recycles = "auto"
  recycle_early_stop_tolerance = "auto"
  pairing_strategy = "greedy"
  max_msa = "auto"
  num_seeds = 1
  use_dropout = False
  num_recycles = None if num_recycles == "auto" else int(num_recycles)
  recycle_early_stop_tolerance = None if recycle_early_stop_tolerance == "auto" else float(recycle_early_stop_tolerance)
  if max_msa == "auto": max_msa = None
  save_all = False
  save_recycles = False
  save_to_google_drive = False #@param {type:"boolean"}
  dpi = 200
  if save_to_google_drive:
    from pydrive.drive import GoogleDrive
    from pydrive.auth import GoogleAuth
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    print("You are logged into Google Drive and are good to go!")
  #@title Run Prediction
  display_images = False #@param {type:"boolean"}

  import sys
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)
  from Bio import BiopythonDeprecationWarning
  warnings.simplefilter(action='ignore', category=BiopythonDeprecationWarning)
  from pathlib import Path
  from colabfold.download import download_alphafold_params, default_data_dir
  from colabfold.utils import setup_logging
  from colabfold.batch import get_queries, run, set_model_type
  from colabfold.plot import plot_msa_v2

  import os
  import numpy as np

  try:
    K80_chk = os.popen('nvidia-smi | grep "Tesla K80" | wc -l').read()
  except:
    K80_chk = "0"
    pass
  if "1" in K80_chk:
    print("WARNING: found GPU Tesla K80: limited to total length < 1000")
    if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
      del os.environ["TF_FORCE_UNIFIED_MEMORY"]
    if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
      del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]

  from colabfold.colabfold import plot_protein
  from pathlib import Path
  import matplotlib.pyplot as plt

  # For some reason we need that to get pdbfixer to import
  if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
      sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

  def input_features_callback(input_features):
    if display_images:
      plot_msa_v2(input_features)
      plt.show()
      plt.close()

  def prediction_callback(protein_obj, length,
                          prediction_result, input_features, mode):
    model_name, relaxed = mode
    if not relaxed:
      if display_images:
        fig = plot_protein(protein_obj, Ls=length, dpi=150)
        plt.show()
        plt.close()

  result_dir = jobname
  log_filename = os.path.join(jobname,"log.txt")
  if not os.path.isfile(log_filename) or 'logging_setup' not in globals():
    setup_logging(Path(log_filename))
    logging_setup = True

  queries, is_complex = get_queries(queries_path)
  model_type = set_model_type(is_complex, model_type)

  if "multimer" in model_type and max_msa is not None:
    use_cluster_profile = False
  else:
    use_cluster_profile = True

  download_alphafold_params(model_type, Path("."))
  results = run(
      queries=queries,
      result_dir=result_dir,
      use_templates=use_templates,
      custom_template_path=custom_template_path,
      num_relax=num_relax,
      msa_mode=msa_mode,
      model_type=model_type,
      num_models=5,
      num_recycles=num_recycles,
      recycle_early_stop_tolerance=recycle_early_stop_tolerance,
      num_seeds=num_seeds,
      use_dropout=use_dropout,
      model_order=[1,2,3,4,5],
      is_complex=is_complex,
      data_dir=Path("."),
      keep_existing_results=False,
      rank_by="auto",
      pair_mode=pair_mode,
      pairing_strategy=pairing_strategy,
      stop_at_score=float(100),
      prediction_callback=prediction_callback,
      dpi=dpi,
      zip_results=False,
      save_all=save_all,
      max_msa=max_msa,
      use_cluster_profile=use_cluster_profile,
      input_features_callback=input_features_callback,
      save_recycles=save_recycles,
  )
  pdb_file_path = os.path.join(jobname, "test.pdb")
  results_zip = f"{jobname}.result.zip"
  os.system(f"zip -r {results_zip} {jobname}")

  import locale
  def getpreferredencoding(do_setlocale = True):
      return "UTF-8"
  locale.getpreferredencoding = getpreferredencoding
  #e.g.path: /content/test_94f6f_6/test_94f6f_6_unrelaxed_rank_001_alphafold2_ptm_model_2_seed_000.pdb
  pdb_file_path = os.path.join(jobname, jobname+"_unrelaxed_rank_001_alphafold2_ptm_model_2_seed_000.pdb")
  # Install necessary packages using pip
  !pip install -q matplotlib
  !pip install -q numpy
  !pip install -q biopython

  # Import required libraries
  import numpy as np
  import matplotlib as mpl
  import matplotlib.pyplot as plt
  from Bio.SeqUtils import ProtParam
  from Bio.PDB import PDBParser
  from Bio.PDB import DSSP

  #!sudo apt-get install -y dssp
  #os.environ['PATH'] += ":/usr/bin/dssp"
  # Step 1: Install packages and prepare variables.

  # Define a list of all amino acid residues
  all_residues = ['A','B','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','X','Y','Z']

  # Define hydrophobicity values for each amino acid residue
  hydrophobicity_value = [3.9,25.6,8.3,-0.9,-0.9,29.9,0.0,3.4,22.4,-1.1,24.2,16.3,0.5,9.7,0.5,3.9,0.5,3.9,14.4,32.9,29.5,15.4,-1.1]

  # Define hydrophilic residues
  hydrophilic_residue = ['Q','N','S','R','D','H','K','Z','E']

  # Prompt user to enter a sequence
  '''
  sequence_2 = input("Enter sequence:")
  sequence = np.array(list(sequence_2))
  '''
  sequence = np.array(list(query_sequence))
  # Assign lyticity values to each residue in the peptide sequence
  lyticity_assignment = []
  for x in sequence:
    lyticity_assignment.append(hydrophobicity_value[all_residues.index(x)])

  # Create an array of i+4 sums
  i_plus_4_sums = []
  for i in range(len(sequence)-5):
    if sequence[i] not in hydrophilic_residue and sequence[i+4] not in hydrophilic_residue:
      i_plus_4_sums.append(lyticity_assignment[i] + lyticity_assignment[i+4])

  # Create an array of i+3 sums
  i_plus_3_sums = []
  for i in range(len(sequence)-4):
    if sequence[i] not in hydrophilic_residue and sequence[i+3] not in hydrophilic_residue:
      i_plus_3_sums.append(lyticity_assignment[i] + lyticity_assignment[i+3])

  # Calculate the total lyticity value by summing i+4 and i+3 sums
  total_lyticity = sum(i_plus_4_sums) + sum(i_plus_3_sums)

  # Load the protein structure from a PDB file
  parser = PDBParser(QUIET=True)

  try:
    structure = parser.get_structure("protein", pdb_file_path)  # Replace with your PDB file path
  except:
    try:
      pdb_file_path = os.path.join(jobname, jobname+"_unrelaxed_rank_001_alphafold2_ptm_model_3_seed_000.pdb")
      structure = parser.get_structure("protein", pdb_file_path)  # Replace with your PDB file path
    except:
      try:
        pdb_file_path = os.path.join(jobname, jobname+"_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb")
        structure = parser.get_structure("protein", pdb_file_path)  # Replace with your PDB file path
      except:
        try:
          pdb_file_path = os.path.join(jobname, jobname+"_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb")
          structure = parser.get_structure("protein", pdb_file_path)  # Replace with your PDB file path
        except:
          pdb_file_path = os.path.join(jobname, jobname+"_unrelaxed_rank_001_alphafold2_ptm_model_1_seed_000.pdb")
          structure = parser.get_structure("protein", pdb_file_path)  # Replace with your PDB file path
  # Define hydrophobicity values for amino acids in a different format
  hydrophobicity_values_0 = {
      'LLE': 22.4, 'VAL': 14.4, 'LEU': 24.2, 'PHE': 29.9, 'CYS': 8.4,
      'MET': 16.3, 'ALA': 3.9, 'GLY': 0.0, 'THR': 3.9, 'SER': 0.5,
      'TRP': 32.9, 'TYR': 15.4, 'PRO': 9.7, 'HIS': 3.4, 'GLU': -0.9,
      'GLN': 0.5, 'ASP': -0.9, 'ASN': 0.5, 'LYS': -1.1, 'ARG': 3.9
  }

  # Define hydrophobicity values for individual amino acids
  hydrophobicity_values_1 = {
      'A': 3.9, 'B': 25.6, 'C': 8.4, 'D': -0.9, 'E': -0.9,
      'F': 29.9, 'G': 0.0, 'H': 3.4, 'I': 22.4, 'K': -1.1,
      'L': 24.2, 'M': 16.3, 'N': 0.5, 'P': 9.7, 'Q': 0.5,
      'R': 3.9, 'S': 0.5, 'T': 3.9, 'V': 14.4, 'W': 32.9,
      'X': 29.5, 'Y': 15.4, 'Z': -1.1
  }

  # Calculate the total hydrophobicity of the input sequence
  total_hydrophobicity = sum(hydrophobicity_values_1.get(aa, 0) for aa in query_sequence)

  # Initialize variables for the hydrophobic moment vector
  hydrophobic_moment_vector = np.zeros(3)

  # Iterate through the residues in the protein structure
  for model in structure:
      for chain in model:
          for residue in chain:
              amino_acid = residue.get_resname().strip()
              # Check if the amino acid is in the hydrophobicity dictionary
              if amino_acid in hydrophobicity_values_0:
                  hydrophobicity = hydrophobicity_values_0[amino_acid]
                  # Get the coordinates of an appropriate atom (e.g., Cα)
                  try:
                      atom = residue['CA']
                      coordinates = atom.get_coord()
                      # Calculate the contribution to the hydrophobic moment vector
                      hydrophobic_moment_vector += hydrophobicity * coordinates
                  except KeyError:
                      pass  # Handle missing atom in residue, e.g., Glycine

  # Calculate the magnitude of the hydrophobic moment vector
  magnitude = np.linalg.norm(hydrophobic_moment_vector)

  # Normalize the hydrophobic moment vector
  if magnitude > 0:
      normalized_hydrophobic_moment = hydrophobic_moment_vector / magnitude
  else:
      normalized_hydrophobic_moment = np.zeros(3)

  # Create a ProteinAnalysis object for the input sequence
  protein = ProtParam.ProteinAnalysis(query_sequence)

  # Set the pH value for peptide charge calculation
  pH_value = 7.0

  # Calculate the peptide charge at the specified pH
  peptide_charge = protein.charge_at_pH(pH_value)
  TL.append(total_lyticity)
  TH.append(total_hydrophobicity)
  PC.append(peptide_charge)
  HM.append(hydrophobic_moment_vector)
  NHM.append(normalized_hydrophobic_moment)
  return
process=0 #process=? Write the start process index here to see the output state during running
'''
Recursion_Seq(ans[0])
Recursion_Seq(ans[1])
Recursion_Seq(ans[2])
'''
#The program use extremely large RAM, you can split the data and run seperately if you don't have enough RAM

for i in range(0,len(ans)//2):
  process+=1
  print("Starting Process "+str(process))
  Recursion_Seq(ans[i])
  print("Process "+str(process)+" finish")


'''
for i in range(len(ans)//2,len(ans)):
  process+=1
  print("Starting Process "+str(process))
  Recursion_Seq(ans[i])
  print("Process "+str(process)+" finish")
'''


#for item in ans:
#  Recursion_Seq(item)
#  process+=1
#  print(process)


print('DONE')


In [ ]:
#@title Excel Generator
fornow=[]
aa=[]
for i in range(len(ans)//2,len(ans)):
  aa.append(ans[i])


results_df = pd.DataFrame({
    'Sequence': aa,
    'Lyticity Index': TL,
    'Total Hydrophobicity': TH,
    'Peptide Charge at pH 7.0': PC,
    '3D Hydrophobic Moment Vector': HM,
    'Normalized Direction Vector': NHM
})

results_excel_file = 'sequences_results.xlsx'
results_df.to_excel(results_excel_file, index=False)

import os

# Get the current working directory
current_directory = os.getcwd()

# Specify the Excel file name
excel_file_name = 'sequences_results.xlsx'

# Create the full path to the Excel file
excel_file_path = os.path.join(current_directory, excel_file_name)

# Check if the file exists
if os.path.isfile(excel_file_path):
    print(f"The Excel file '{excel_file_name}' is located at: {excel_file_path}")
else:
    print(f"The Excel file '{excel_file_name}' was not found in the current directory.")
